### Build SVM model for 20,000 images

In [48]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import svm
import time

In [2]:
# define Normalize function
def l2_normalize(x):
    return x / np.sqrt(np.sum(np.multiply(x, x)))

In [3]:
'''basedir = '../../../../../../../Desktop/Face_recognition/'
datasetpath = '../../../../../../../Desktop/Face_recognition/CelebA/Training.txt'
df_training = pd.read_csv(datasetpath, sep=" ", index_col = False, names=('image', 'class'))
df_training'''


,image,class
0,CelebA/img_align_celeba/158477.jpg,14
1,CelebA/img_align_celeba/122873.jpg,14
2,CelebA/img_align_celeba/101078.jpg,14
3,CelebA/img_align_celeba/122060.jpg,14
4,CelebA/img_align_celeba/066818.jpg,14
...,...,...
6995,CelebA/img_align_celeba/095884.jpg,1731
6996,CelebA/img_align_celeba/103866.jpg,1731
6997,CelebA/img_align_celeba/135771.jpg,1731
6998,CelebA/img_align_celeba/155794.jpg,1731


In [4]:
'''datasetpath = '../../../../../../../Desktop/Face_recognition/CelebA/Training_1.txt'
df_training_1 = pd.read_csv(datasetpath, sep=" ", index_col = False, names=('image', 'class'))
df_training_1'''


,image,class
0,CelebA/img_align_celeba/063174.jpg,1731
1,CelebA/img_align_celeba/112721.jpg,1731
2,CelebA/img_align_celeba/026885.jpg,1731
3,CelebA/img_align_celeba/151917.jpg,1731
4,CelebA/img_align_celeba/105429.jpg,1731
...,...,...
6995,CelebA/img_align_celeba/042578.jpg,3184
6996,CelebA/img_align_celeba/005486.jpg,3184
6997,CelebA/img_align_celeba/009136.jpg,3184
6998,CelebA/img_align_celeba/040929.jpg,3184


In [5]:
# read excel file
df_emd = pd.read_excel("CelebA.xlsx", sheet_name = "Sheet1")
df_emd

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
0,../../../../../../../Desktop/Face_recognition/...,14,-0.111844,-0.040397,0.008005,-0.009808,0.053051,-0.018495,0.001613,-0.010849,...,-0.022299,-0.026468,0.015961,0.012991,0.065044,-0.042305,0.014783,-0.034854,0.005985,0.065682
1,../../../../../../../Desktop/Face_recognition/...,14,-0.053899,-0.027015,0.073401,-0.019615,0.023837,0.003803,0.010742,-0.056180,...,0.000665,0.012398,-0.008948,-0.000255,0.079757,-0.022045,-0.024136,-0.005053,-0.031327,0.061850
2,../../../../../../../Desktop/Face_recognition/...,14,-0.093583,0.010288,0.011210,0.008576,0.065813,-0.036413,-0.011279,-0.011245,...,0.015279,-0.042507,0.043787,0.029044,0.044691,-0.031127,0.026636,0.015283,-0.022243,0.022617
3,../../../../../../../Desktop/Face_recognition/...,14,-0.030548,-0.018362,0.009033,-0.040010,0.031385,0.008026,0.022224,-0.000514,...,-0.030774,0.008374,-0.029686,-0.002855,0.121509,-0.031774,-0.024131,-0.072375,0.047133,0.035932
4,../../../../../../../Desktop/Face_recognition/...,14,-0.083355,-0.050736,-0.018019,0.013635,0.052849,0.004634,0.016608,-0.043881,...,0.056371,-0.033615,0.012772,0.050218,0.072544,0.020858,-0.011089,-0.073444,-0.004395,0.017994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,../../../../../../../Desktop/Face_recognition/...,1726,0.004924,-0.054827,-0.017430,-0.002137,0.110399,0.019909,-0.008500,-0.085542,...,-0.037298,0.018752,0.024837,0.043310,0.075285,-0.001737,-0.061697,-0.007269,0.055907,-0.105362
9996,../../../../../../../Desktop/Face_recognition/...,1726,0.002935,-0.028735,0.009888,-0.003222,0.050696,0.033102,0.032420,-0.072258,...,0.025078,-0.006978,0.044163,0.102358,0.059551,-0.009940,0.013006,-0.007557,-0.041904,-0.093556
9997,../../../../../../../Desktop/Face_recognition/...,1731,-0.027912,0.048129,0.004471,0.003006,0.016043,-0.018855,0.025526,0.053270,...,-0.004781,-0.045373,-0.044482,-0.075896,-0.000770,0.020550,0.054452,0.019588,0.041638,0.005107
9998,../../../../../../../Desktop/Face_recognition/...,1731,-0.016202,0.053886,0.014018,-0.003342,0.031734,-0.098537,0.004029,0.034217,...,-0.007177,-0.048307,-0.040087,-0.075403,-0.054230,0.028954,0.039627,0.002413,0.056650,0.000439


In [6]:
df_emd_1 = pd.read_excel("CelebA_1.xlsx", sheet_name = "Sheet1")
df_emd_1

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
0,../../../../../../../Desktop/Face_recognition/...,1731,0.021669,0.002306,0.026310,0.050295,0.038290,0.009720,0.029378,0.077409,...,0.018958,-0.030904,-0.032928,-0.060037,0.016933,0.014269,-0.006921,-0.001624,-0.031213,-0.034540
1,../../../../../../../Desktop/Face_recognition/...,1731,0.019007,0.092497,0.064539,0.063037,-0.028907,0.005799,-0.030318,0.078422,...,0.011086,-0.086196,-0.074726,-0.055394,0.058731,0.046872,0.041321,0.024605,0.047105,0.012780
2,../../../../../../../Desktop/Face_recognition/...,1731,-0.029977,0.077251,0.018499,-0.003051,-0.042731,-0.054179,-0.001995,0.096655,...,-0.026730,-0.014843,-0.045539,-0.053452,-0.000913,0.053057,0.078197,0.032263,0.034662,-0.066154
3,../../../../../../../Desktop/Face_recognition/...,1731,0.028536,0.001082,-0.042037,-0.038239,-0.014289,-0.050447,0.057441,0.068619,...,-0.013301,-0.046130,-0.019959,-0.050068,-0.059313,0.047489,0.000945,-0.024756,-0.007010,-0.013068
4,../../../../../../../Desktop/Face_recognition/...,1731,-0.054946,0.058223,0.060745,0.087490,-0.001162,-0.035454,0.023205,0.050554,...,0.095568,0.017767,-0.009459,0.000451,0.056613,0.020473,0.096778,0.011624,-0.060036,-0.017033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,../../../../../../../Desktop/Face_recognition/...,3184,0.043901,-0.028672,0.066481,-0.022024,-0.018030,0.088724,-0.072438,-0.094951,...,0.037053,0.037722,0.111516,-0.005865,-0.039585,-0.001362,-0.030260,-0.067203,-0.063776,0.000614
9996,../../../../../../../Desktop/Face_recognition/...,3184,0.019330,0.061899,-0.027057,0.006853,-0.092524,0.105420,-0.017241,-0.012448,...,0.080704,0.095147,0.030326,0.017269,0.061613,-0.016260,-0.012771,-0.004708,-0.071226,-0.019530
9997,../../../../../../../Desktop/Face_recognition/...,3184,0.045204,-0.049184,-0.055105,-0.051389,-0.066671,-0.006861,-0.022320,-0.027259,...,0.019934,0.098106,0.053529,0.062384,-0.021278,0.061815,-0.057003,-0.084106,-0.081037,-0.071945
9998,../../../../../../../Desktop/Face_recognition/...,3184,0.007905,-0.017172,-0.044709,-0.001010,-0.057629,0.063645,0.004024,-0.035885,...,0.055910,0.099537,0.078997,0.077398,0.010229,0.006133,-0.009962,-0.028951,-0.050378,-0.062757


In [32]:
len(set(list(df_emd_1["Labels"])))

334

In [7]:
df_train = df_emd.head(7000)
df_train

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
0,../../../../../../../Desktop/Face_recognition/...,14,-0.111844,-0.040397,0.008005,-0.009808,0.053051,-0.018495,0.001613,-0.010849,...,-0.022299,-0.026468,0.015961,0.012991,0.065044,-0.042305,0.014783,-0.034854,0.005985,0.065682
1,../../../../../../../Desktop/Face_recognition/...,14,-0.053899,-0.027015,0.073401,-0.019615,0.023837,0.003803,0.010742,-0.056180,...,0.000665,0.012398,-0.008948,-0.000255,0.079757,-0.022045,-0.024136,-0.005053,-0.031327,0.061850
2,../../../../../../../Desktop/Face_recognition/...,14,-0.093583,0.010288,0.011210,0.008576,0.065813,-0.036413,-0.011279,-0.011245,...,0.015279,-0.042507,0.043787,0.029044,0.044691,-0.031127,0.026636,0.015283,-0.022243,0.022617
3,../../../../../../../Desktop/Face_recognition/...,14,-0.030548,-0.018362,0.009033,-0.040010,0.031385,0.008026,0.022224,-0.000514,...,-0.030774,0.008374,-0.029686,-0.002855,0.121509,-0.031774,-0.024131,-0.072375,0.047133,0.035932
4,../../../../../../../Desktop/Face_recognition/...,14,-0.083355,-0.050736,-0.018019,0.013635,0.052849,0.004634,0.016608,-0.043881,...,0.056371,-0.033615,0.012772,0.050218,0.072544,0.020858,-0.011089,-0.073444,-0.004395,0.017994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,../../../../../../../Desktop/Face_recognition/...,1731,-0.014745,0.022017,0.016941,0.025487,-0.006760,-0.027176,0.059529,0.035218,...,0.003477,-0.026958,-0.082625,-0.066671,-0.020683,0.061718,0.044349,0.040125,0.035547,-0.026744
6996,../../../../../../../Desktop/Face_recognition/...,1731,-0.003253,0.079098,0.015410,-0.001021,0.031017,-0.048389,-0.042207,0.060792,...,-0.000819,-0.061776,-0.070643,-0.079764,0.027629,0.006421,-0.001172,0.005584,0.090907,0.047774
6997,../../../../../../../Desktop/Face_recognition/...,1731,0.030423,0.045842,-0.009864,0.015251,0.025280,-0.070754,-0.038140,0.056707,...,-0.004806,-0.041573,-0.043516,-0.107706,-0.033838,0.039708,0.048379,0.032396,0.075916,-0.039056
6998,../../../../../../../Desktop/Face_recognition/...,1731,-0.005930,0.039172,0.019138,-0.014550,0.019051,-0.070911,0.010818,0.045626,...,0.009604,-0.080336,-0.038013,-0.066373,-0.010019,0.035545,0.052552,0.050478,0.060382,0.008987


In [8]:
df_train_1 = df_emd_1.iloc[:7000,]
df_train_1

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
0,../../../../../../../Desktop/Face_recognition/...,1731,0.021669,0.002306,0.026310,0.050295,0.038290,0.009720,0.029378,0.077409,...,0.018958,-0.030904,-0.032928,-0.060037,0.016933,0.014269,-0.006921,-0.001624,-0.031213,-0.034540
1,../../../../../../../Desktop/Face_recognition/...,1731,0.019007,0.092497,0.064539,0.063037,-0.028907,0.005799,-0.030318,0.078422,...,0.011086,-0.086196,-0.074726,-0.055394,0.058731,0.046872,0.041321,0.024605,0.047105,0.012780
2,../../../../../../../Desktop/Face_recognition/...,1731,-0.029977,0.077251,0.018499,-0.003051,-0.042731,-0.054179,-0.001995,0.096655,...,-0.026730,-0.014843,-0.045539,-0.053452,-0.000913,0.053057,0.078197,0.032263,0.034662,-0.066154
3,../../../../../../../Desktop/Face_recognition/...,1731,0.028536,0.001082,-0.042037,-0.038239,-0.014289,-0.050447,0.057441,0.068619,...,-0.013301,-0.046130,-0.019959,-0.050068,-0.059313,0.047489,0.000945,-0.024756,-0.007010,-0.013068
4,../../../../../../../Desktop/Face_recognition/...,1731,-0.054946,0.058223,0.060745,0.087490,-0.001162,-0.035454,0.023205,0.050554,...,0.095568,0.017767,-0.009459,0.000451,0.056613,0.020473,0.096778,0.011624,-0.060036,-0.017033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,../../../../../../../Desktop/Face_recognition/...,3184,0.038555,-0.015641,0.041043,-0.036318,-0.097194,0.076910,-0.047347,-0.038111,...,-0.002504,0.103243,0.046312,0.030563,-0.045588,-0.025483,0.004140,-0.074358,-0.075892,-0.036838
6996,../../../../../../../Desktop/Face_recognition/...,3184,0.002589,0.001258,0.038455,-0.044394,-0.066031,0.042881,-0.053122,-0.079258,...,0.077865,0.083681,0.089423,0.006674,-0.061378,0.007846,-0.028989,-0.050718,-0.059393,-0.056936
6997,../../../../../../../Desktop/Face_recognition/...,3184,0.010246,0.039373,0.075980,-0.070188,-0.054329,0.047888,-0.079921,-0.058255,...,0.033470,0.058836,0.061232,-0.008481,-0.070466,-0.018707,-0.039262,-0.007708,-0.049795,-0.034092
6998,../../../../../../../Desktop/Face_recognition/...,3184,0.101193,0.003422,-0.000994,-0.021710,-0.132490,0.045452,-0.026736,-0.025297,...,0.015471,0.100872,0.075284,0.015941,-0.036921,0.041485,-0.040028,-0.054802,-0.074290,-0.081896


In [9]:
# combine the training excel file together
df_train_2 = pd.concat([df_train, df_train_1])
df_train_2

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
0,../../../../../../../Desktop/Face_recognition/...,14,-0.111844,-0.040397,0.008005,-0.009808,0.053051,-0.018495,0.001613,-0.010849,...,-0.022299,-0.026468,0.015961,0.012991,0.065044,-0.042305,0.014783,-0.034854,0.005985,0.065682
1,../../../../../../../Desktop/Face_recognition/...,14,-0.053899,-0.027015,0.073401,-0.019615,0.023837,0.003803,0.010742,-0.056180,...,0.000665,0.012398,-0.008948,-0.000255,0.079757,-0.022045,-0.024136,-0.005053,-0.031327,0.061850
2,../../../../../../../Desktop/Face_recognition/...,14,-0.093583,0.010288,0.011210,0.008576,0.065813,-0.036413,-0.011279,-0.011245,...,0.015279,-0.042507,0.043787,0.029044,0.044691,-0.031127,0.026636,0.015283,-0.022243,0.022617
3,../../../../../../../Desktop/Face_recognition/...,14,-0.030548,-0.018362,0.009033,-0.040010,0.031385,0.008026,0.022224,-0.000514,...,-0.030774,0.008374,-0.029686,-0.002855,0.121509,-0.031774,-0.024131,-0.072375,0.047133,0.035932
4,../../../../../../../Desktop/Face_recognition/...,14,-0.083355,-0.050736,-0.018019,0.013635,0.052849,0.004634,0.016608,-0.043881,...,0.056371,-0.033615,0.012772,0.050218,0.072544,0.020858,-0.011089,-0.073444,-0.004395,0.017994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,../../../../../../../Desktop/Face_recognition/...,3184,0.038555,-0.015641,0.041043,-0.036318,-0.097194,0.076910,-0.047347,-0.038111,...,-0.002504,0.103243,0.046312,0.030563,-0.045588,-0.025483,0.004140,-0.074358,-0.075892,-0.036838
6996,../../../../../../../Desktop/Face_recognition/...,3184,0.002589,0.001258,0.038455,-0.044394,-0.066031,0.042881,-0.053122,-0.079258,...,0.077865,0.083681,0.089423,0.006674,-0.061378,0.007846,-0.028989,-0.050718,-0.059393,-0.056936
6997,../../../../../../../Desktop/Face_recognition/...,3184,0.010246,0.039373,0.075980,-0.070188,-0.054329,0.047888,-0.079921,-0.058255,...,0.033470,0.058836,0.061232,-0.008481,-0.070466,-0.018707,-0.039262,-0.007708,-0.049795,-0.034092
6998,../../../../../../../Desktop/Face_recognition/...,3184,0.101193,0.003422,-0.000994,-0.021710,-0.132490,0.045452,-0.026736,-0.025297,...,0.015471,0.100872,0.075284,0.015941,-0.036921,0.041485,-0.040028,-0.054802,-0.074290,-0.081896


In [10]:
df_test =df_emd.tail(3000)
df_test

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
7000,../../../../../../../Desktop/Face_recognition/...,14,-0.085909,-0.002860,0.046347,-0.000180,0.048312,-0.038898,0.036218,-0.039623,...,-0.012955,0.065656,-0.022992,0.019879,0.032438,0.035387,-0.004797,-0.034639,-0.030574,-0.018152
7001,../../../../../../../Desktop/Face_recognition/...,14,-0.089407,0.025720,-0.005736,-0.011605,0.070474,0.010601,-0.043271,-0.006145,...,0.055619,-0.025066,0.094652,0.000660,0.029296,0.023424,-0.018014,-0.058202,0.025183,-0.011439
7002,../../../../../../../Desktop/Face_recognition/...,14,-0.035921,0.035207,0.069836,-0.017619,0.068801,-0.029043,0.005936,-0.076082,...,-0.014212,-0.027021,0.019310,-0.011342,0.012175,0.048851,0.013080,-0.018034,0.055073,0.004470
7003,../../../../../../../Desktop/Face_recognition/...,14,-0.090283,0.007377,0.029885,0.016015,0.045127,-0.010867,-0.027002,-0.037835,...,0.020522,-0.043777,0.065696,-0.020636,0.023398,-0.015565,0.021063,-0.026025,0.009466,0.037267
7004,../../../../../../../Desktop/Face_recognition/...,14,-0.095194,-0.006736,0.006922,-0.004694,0.073630,0.005073,0.035674,-0.010256,...,0.019507,-0.021528,-0.005447,-0.031595,0.046878,0.061852,0.049048,-0.039904,-0.033685,0.003591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,../../../../../../../Desktop/Face_recognition/...,1726,0.004924,-0.054827,-0.017430,-0.002137,0.110399,0.019909,-0.008500,-0.085542,...,-0.037298,0.018752,0.024837,0.043310,0.075285,-0.001737,-0.061697,-0.007269,0.055907,-0.105362
9996,../../../../../../../Desktop/Face_recognition/...,1726,0.002935,-0.028735,0.009888,-0.003222,0.050696,0.033102,0.032420,-0.072258,...,0.025078,-0.006978,0.044163,0.102358,0.059551,-0.009940,0.013006,-0.007557,-0.041904,-0.093556
9997,../../../../../../../Desktop/Face_recognition/...,1731,-0.027912,0.048129,0.004471,0.003006,0.016043,-0.018855,0.025526,0.053270,...,-0.004781,-0.045373,-0.044482,-0.075896,-0.000770,0.020550,0.054452,0.019588,0.041638,0.005107
9998,../../../../../../../Desktop/Face_recognition/...,1731,-0.016202,0.053886,0.014018,-0.003342,0.031734,-0.098537,0.004029,0.034217,...,-0.007177,-0.048307,-0.040087,-0.075403,-0.054230,0.028954,0.039627,0.002413,0.056650,0.000439


In [11]:
df_test_1 =df_emd_1.iloc[7000:10000,]
df_test_1

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
7000,../../../../../../../Desktop/Face_recognition/...,1731,0.003453,0.071780,0.038619,0.022855,0.001358,-0.030596,0.028463,0.073300,...,0.015929,-0.028337,-0.073420,-0.052397,0.051907,0.047692,0.063419,0.059815,0.047995,-0.025502
7001,../../../../../../../Desktop/Face_recognition/...,1731,0.004959,0.033958,0.009409,-0.018046,-0.006176,-0.048943,0.036075,0.048462,...,0.007098,-0.056783,0.012240,-0.058282,-0.041870,0.002556,0.043908,-0.007389,0.044053,-0.039894
7002,../../../../../../../Desktop/Face_recognition/...,1731,-0.031544,0.049742,-0.004000,0.046248,0.029388,-0.042167,0.015940,0.040225,...,0.049282,-0.055584,-0.024051,-0.058634,-0.013916,0.036101,0.054970,0.004811,0.076681,-0.000637
7003,../../../../../../../Desktop/Face_recognition/...,1731,-0.034137,0.075226,0.051737,0.019003,-0.062853,0.022240,-0.039892,0.030000,...,-0.025934,0.001536,-0.099888,-0.122344,0.020429,-0.005915,0.048773,0.079400,0.046712,0.061352
7004,../../../../../../../Desktop/Face_recognition/...,1731,0.014296,0.029437,0.048153,0.038834,-0.001077,-0.045014,0.019673,0.063182,...,-0.015812,-0.058249,-0.015218,-0.087772,-0.041262,0.041881,0.019473,0.029036,0.021662,0.003586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,../../../../../../../Desktop/Face_recognition/...,3184,0.043901,-0.028672,0.066481,-0.022024,-0.018030,0.088724,-0.072438,-0.094951,...,0.037053,0.037722,0.111516,-0.005865,-0.039585,-0.001362,-0.030260,-0.067203,-0.063776,0.000614
9996,../../../../../../../Desktop/Face_recognition/...,3184,0.019330,0.061899,-0.027057,0.006853,-0.092524,0.105420,-0.017241,-0.012448,...,0.080704,0.095147,0.030326,0.017269,0.061613,-0.016260,-0.012771,-0.004708,-0.071226,-0.019530
9997,../../../../../../../Desktop/Face_recognition/...,3184,0.045204,-0.049184,-0.055105,-0.051389,-0.066671,-0.006861,-0.022320,-0.027259,...,0.019934,0.098106,0.053529,0.062384,-0.021278,0.061815,-0.057003,-0.084106,-0.081037,-0.071945
9998,../../../../../../../Desktop/Face_recognition/...,3184,0.007905,-0.017172,-0.044709,-0.001010,-0.057629,0.063645,0.004024,-0.035885,...,0.055910,0.099537,0.078997,0.077398,0.010229,0.006133,-0.009962,-0.028951,-0.050378,-0.062757


In [12]:
# combine testinf excel file together
df_test_2 = pd.concat([df_test, df_test_1])
df_test_2

,Images,Labels,1,2,3,4,5,6,7,8,...,503,504,505,506,507,508,509,510,511,512
7000,../../../../../../../Desktop/Face_recognition/...,14,-0.085909,-0.002860,0.046347,-0.000180,0.048312,-0.038898,0.036218,-0.039623,...,-0.012955,0.065656,-0.022992,0.019879,0.032438,0.035387,-0.004797,-0.034639,-0.030574,-0.018152
7001,../../../../../../../Desktop/Face_recognition/...,14,-0.089407,0.025720,-0.005736,-0.011605,0.070474,0.010601,-0.043271,-0.006145,...,0.055619,-0.025066,0.094652,0.000660,0.029296,0.023424,-0.018014,-0.058202,0.025183,-0.011439
7002,../../../../../../../Desktop/Face_recognition/...,14,-0.035921,0.035207,0.069836,-0.017619,0.068801,-0.029043,0.005936,-0.076082,...,-0.014212,-0.027021,0.019310,-0.011342,0.012175,0.048851,0.013080,-0.018034,0.055073,0.004470
7003,../../../../../../../Desktop/Face_recognition/...,14,-0.090283,0.007377,0.029885,0.016015,0.045127,-0.010867,-0.027002,-0.037835,...,0.020522,-0.043777,0.065696,-0.020636,0.023398,-0.015565,0.021063,-0.026025,0.009466,0.037267
7004,../../../../../../../Desktop/Face_recognition/...,14,-0.095194,-0.006736,0.006922,-0.004694,0.073630,0.005073,0.035674,-0.010256,...,0.019507,-0.021528,-0.005447,-0.031595,0.046878,0.061852,0.049048,-0.039904,-0.033685,0.003591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,../../../../../../../Desktop/Face_recognition/...,3184,0.043901,-0.028672,0.066481,-0.022024,-0.018030,0.088724,-0.072438,-0.094951,...,0.037053,0.037722,0.111516,-0.005865,-0.039585,-0.001362,-0.030260,-0.067203,-0.063776,0.000614
9996,../../../../../../../Desktop/Face_recognition/...,3184,0.019330,0.061899,-0.027057,0.006853,-0.092524,0.105420,-0.017241,-0.012448,...,0.080704,0.095147,0.030326,0.017269,0.061613,-0.016260,-0.012771,-0.004708,-0.071226,-0.019530
9997,../../../../../../../Desktop/Face_recognition/...,3184,0.045204,-0.049184,-0.055105,-0.051389,-0.066671,-0.006861,-0.022320,-0.027259,...,0.019934,0.098106,0.053529,0.062384,-0.021278,0.061815,-0.057003,-0.084106,-0.081037,-0.071945
9998,../../../../../../../Desktop/Face_recognition/...,3184,0.007905,-0.017172,-0.044709,-0.001010,-0.057629,0.063645,0.004024,-0.035885,...,0.055910,0.099537,0.078997,0.077398,0.010229,0.006133,-0.009962,-0.028951,-0.050378,-0.062757


In [13]:
#df_emd
#df_emd.drop(df_emd.head(7000).index, inplace = True)
#df_emd
#new_df.loc[len(new_df)] = df_emd.iloc[30]

In [14]:

'''for column in feature_names:
    print ("1-------"+ str(column))
    print (df_emd[column].value_counts(dropna=False))'''

'''sentence = 'CelebA/img_align_celeba/115855.jpg'
sentence.rsplit('CelebA/img_align_celeba/', 1)[1]

'''


'''for value in df_training['image']:
    img_train = value.rsplit('CelebA/img_align_celeba/', 1)[1]
    for index, row_emd in df_emd.iterrows():
        img_emd = row_emd['Images'].rsplit('../../../../../../../Desktop/Face_recognition/CelebA/img_align_celeba/', 1)[1]
        
        if(img_train == img_emd):
            print("emd   " + img_emd)
            print("train " + img_train)
            #index_names = new_df[new_df['Images'] == row_emd['Images']].index
            #print(index_names)
            new_df.loc[len(new_df)] = row_emd
            print(len(new_df))'''
            
        


'for value in df_training[\'image\']:\n    img_train = value.rsplit(\'CelebA/img_align_celeba/\', 1)[1]\n    for index, row_emd in df_emd.iterrows():\n        img_emd = row_emd[\'Images\'].rsplit(\'../../../../../../../Desktop/Face_recognition/CelebA/img_align_celeba/\', 1)[1]\n        \n        if(img_train == img_emd):\n            print("emd   " + img_emd)\n            print("train " + img_train)\n            #index_names = new_df[new_df[\'Images\'] == row_emd[\'Images\']].index\n            #print(index_names)\n            new_df.loc[len(new_df)] = row_emd\n            print(len(new_df))'

In [15]:
new_train = df_train_2.copy()
new_test = df_test_2.copy()

In [16]:
feature_names_train = new_train.columns.tolist()
features_train = new_train.iloc[:,2:]
features_train

,1,2,3,4,5,6,7,8,9,10,...,503,504,505,506,507,508,509,510,511,512
0,-0.111844,-0.040397,0.008005,-0.009808,0.053051,-0.018495,0.001613,-0.010849,0.042112,-0.006685,...,-0.022299,-0.026468,0.015961,0.012991,0.065044,-0.042305,0.014783,-0.034854,0.005985,0.065682
1,-0.053899,-0.027015,0.073401,-0.019615,0.023837,0.003803,0.010742,-0.056180,0.010814,0.032049,...,0.000665,0.012398,-0.008948,-0.000255,0.079757,-0.022045,-0.024136,-0.005053,-0.031327,0.061850
2,-0.093583,0.010288,0.011210,0.008576,0.065813,-0.036413,-0.011279,-0.011245,0.101915,-0.049842,...,0.015279,-0.042507,0.043787,0.029044,0.044691,-0.031127,0.026636,0.015283,-0.022243,0.022617
3,-0.030548,-0.018362,0.009033,-0.040010,0.031385,0.008026,0.022224,-0.000514,-0.058607,0.010853,...,-0.030774,0.008374,-0.029686,-0.002855,0.121509,-0.031774,-0.024131,-0.072375,0.047133,0.035932
4,-0.083355,-0.050736,-0.018019,0.013635,0.052849,0.004634,0.016608,-0.043881,-0.023469,0.086539,...,0.056371,-0.033615,0.012772,0.050218,0.072544,0.020858,-0.011089,-0.073444,-0.004395,0.017994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0.038555,-0.015641,0.041043,-0.036318,-0.097194,0.076910,-0.047347,-0.038111,0.031807,-0.005200,...,-0.002504,0.103243,0.046312,0.030563,-0.045588,-0.025483,0.004140,-0.074358,-0.075892,-0.036838
6996,0.002589,0.001258,0.038455,-0.044394,-0.066031,0.042881,-0.053122,-0.079258,0.045547,-0.010908,...,0.077865,0.083681,0.089423,0.006674,-0.061378,0.007846,-0.028989,-0.050718,-0.059393,-0.056936
6997,0.010246,0.039373,0.075980,-0.070188,-0.054329,0.047888,-0.079921,-0.058255,0.019323,-0.006349,...,0.033470,0.058836,0.061232,-0.008481,-0.070466,-0.018707,-0.039262,-0.007708,-0.049795,-0.034092
6998,0.101193,0.003422,-0.000994,-0.021710,-0.132490,0.045452,-0.026736,-0.025297,-0.027953,-0.037279,...,0.015471,0.100872,0.075284,0.015941,-0.036921,0.041485,-0.040028,-0.054802,-0.074290,-0.081896


In [31]:
features_train.sum(axis=1)

0       0.460514
1      -0.503843
2       0.758933
3      -0.523623
4      -0.165257
          ...   
6995    1.150897
6996    0.010422
6997    0.256771
6998    0.839021
6999   -0.636614
Length: 14000, dtype: float64

In [43]:
# find the shape of dataset
val=np.array(features_train)
val.shape

(14000, 512)

In [44]:
val = np.square(val)
val.shape

(14000, 512)

In [45]:
val.shape

(14000, 512)

In [46]:
val = np.sum(val, axis=1)
val

array([1.00000009, 1.00000028, 0.99999999, ..., 0.99999994, 1.00000017,
       0.99999999])

In [17]:
feature_names_train = new_train.columns.tolist()
features_train = new_train.iloc[:,2:]
#features_train = new_train.drop('Labels', 1)
#features_train = new_train.iloc[:,0]
labels_train = new_train['Labels']

In [18]:
feature_names_test = new_test.columns.tolist()
features_test = new_test.iloc[:,2:]
#features_test = new_test.iloc[:,0]
labels_test = new_test['Labels']

In [19]:
# prepere data ready for model
X_train = features_train
y_train = labels_train
X_test = features_test
y_test = labels_test

In [20]:
l2_normalize(X_train)

,1,2,3,4,5,6,7,8,9,10,...,503,504,505,506,507,508,509,510,511,512
0,-0.022207,-0.008161,0.001543,-0.001968,0.010073,-0.003345,0.000324,-0.002047,0.008187,-0.001334,...,-0.004486,-0.005390,0.002937,0.002360,0.012004,-0.009053,0.002859,-0.005403,0.001072,0.012293
1,-0.010702,-0.005458,0.014146,-0.003935,0.004526,0.000688,0.002158,-0.010598,0.002102,0.006394,...,0.000134,0.002525,-0.001646,-0.000046,0.014720,-0.004718,-0.004667,-0.000783,-0.005611,0.011576
2,-0.018582,0.002079,0.002160,0.001721,0.012496,-0.006586,-0.002266,-0.002121,0.019814,-0.009944,...,0.003074,-0.008656,0.008057,0.005277,0.008248,-0.006661,0.005151,0.002369,-0.003984,0.004233
3,-0.006065,-0.003710,0.001741,-0.008028,0.005959,0.001452,0.004466,-0.000097,-0.011394,0.002165,...,-0.006191,0.001705,-0.005462,-0.000519,0.022425,-0.006799,-0.004666,-0.011220,0.008442,0.006725
4,-0.016551,-0.010250,-0.003473,0.002736,0.010034,0.000838,0.003337,-0.008278,-0.004563,0.017265,...,0.011340,-0.006846,0.002350,0.009124,0.013389,0.004464,-0.002144,-0.011386,-0.000787,0.003368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0.007655,-0.003160,0.007910,-0.007287,-0.018454,0.013911,-0.009514,-0.007189,0.006184,-0.001037,...,-0.000504,0.021025,0.008521,0.005553,-0.008414,-0.005453,0.000801,-0.011528,-0.013593,-0.006894
6996,0.000514,0.000254,0.007411,-0.008907,-0.012537,0.007756,-0.010674,-0.014951,0.008855,-0.002176,...,0.015664,0.017041,0.016454,0.001213,-0.011328,0.001679,-0.005606,-0.007863,-0.010638,-0.010656
6997,0.002034,0.007954,0.014643,-0.014082,-0.010315,0.008662,-0.016059,-0.010989,0.003757,-0.001267,...,0.006733,0.011982,0.011266,-0.001541,-0.013005,-0.004003,-0.007592,-0.001195,-0.008919,-0.006380
6998,0.020093,0.000691,-0.000192,-0.004356,-0.025155,0.008221,-0.005372,-0.004772,-0.005434,-0.007437,...,0.003112,0.020542,0.013852,0.002896,-0.006814,0.008877,-0.007740,-0.008496,-0.013306,-0.015327


In [21]:
# call normalize function
l2_normalize(X_test)

,1,2,3,4,5,6,7,8,9,10,...,503,504,505,506,507,508,509,510,511,512
7000,-0.025889,-0.000888,0.013646,-0.000056,0.014170,-0.010767,0.011124,-0.011414,0.001299,0.017088,...,-0.003962,0.020268,-0.006414,0.005536,0.009068,0.011376,-0.001447,-0.008143,-0.008371,-0.005197
7001,-0.026944,0.007983,-0.001689,-0.003590,0.020670,0.002934,-0.013290,-0.001770,0.002908,0.018548,...,0.017008,-0.007738,0.026404,0.000184,0.008190,0.007530,-0.005435,-0.013682,0.006895,-0.003275
7002,-0.010825,0.010927,0.020562,-0.005450,0.020179,-0.008039,0.001823,-0.021917,0.005794,0.020197,...,-0.004346,-0.008341,0.005387,-0.003159,0.003404,0.015704,0.003946,-0.004239,0.015079,0.001280
7003,-0.027208,0.002290,0.008799,0.004954,0.013236,-0.003008,-0.008293,-0.010899,0.017103,0.005653,...,0.006275,-0.013514,0.018326,-0.005747,0.006541,-0.005003,0.006355,-0.006118,0.002592,0.010669
7004,-0.028688,-0.002091,0.002038,-0.001452,0.021595,0.001404,0.010956,-0.002955,0.005407,0.015763,...,0.005965,-0.006646,-0.001520,-0.008799,0.013105,0.019883,0.014798,-0.009380,-0.009223,0.001028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.013230,-0.008899,0.019575,-0.006813,-0.005288,0.024559,-0.022248,-0.027352,0.020670,-0.007559,...,0.011330,0.011645,0.031108,-0.001633,-0.011066,-0.000438,-0.009130,-0.015797,-0.017462,0.000176
9996,0.005825,0.019211,-0.007967,0.002120,-0.027137,0.029181,-0.005295,-0.003586,-0.004325,-0.018041,...,0.024678,0.029371,0.008460,0.004809,0.017224,-0.005227,-0.003853,-0.001107,-0.019502,-0.005591
9997,0.013623,-0.015265,-0.016225,-0.015897,-0.019554,-0.001899,-0.006855,-0.007852,-0.003929,0.000581,...,0.006096,0.030285,0.014933,0.017374,-0.005948,0.019871,-0.017198,-0.019771,-0.022188,-0.020597
9998,0.002382,-0.005330,-0.013164,-0.000313,-0.016902,0.017617,0.001236,-0.010337,-0.005257,-0.005850,...,0.017097,0.030726,0.022037,0.021556,0.002860,0.001972,-0.003006,-0.006806,-0.013793,-0.017967


In [22]:
'''from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
imputer.fit(X) 
X = imputer.transform(X)'''

"from sklearn.preprocessing import Imputer\nimputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)\nimputer.fit(X) \nX = imputer.transform(X)"

In [23]:
'''# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test'''

'# Import train_test_split function\nfrom sklearn.model_selection import train_test_split\n\n# Split dataset into training set and test set\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test'

In [49]:
#Create a svm Classifier
clf = svm.SVC(kernel='rbf', gamma='auto')

start = time.time()

#Train the model using the training sets
clf.fit(X_train, y_train)

end = time.time()

print(f"Runtime of the program is {end - start}")

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Runtime of the program is 520.7372281551361
Accuracy: 0.8731666666666666


In [25]:
#Create a svm Classifier
clf = svm.SVC(kernel='linear', gamma='auto')

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9041666666666667


In [26]:
#Create a svm Classifier
clf = svm.SVC(kernel='poly', gamma='auto')

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8653333333333333


In [27]:
#Create a svm Classifier
clf = svm.SVC(kernel='sigmoid', gamma='auto')

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8728333333333333
